In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
data = pd.read_csv("dataset_edited.csv", encoding="latin1")
data = data.drop(['POS'], axis =1)
data = data.fillna(method="ffill")
data.tail(12)
data['Word'] = data['Word'].str.lower()
data['Tag'] = data['Tag'].str.lower()
print(data['Tag'])
words = set(list(data['Word'].values))
words.add('PADword')
n_words = len(words)
n_words

In [ ]:
tags = list(set(data["Tag"].values))
n_tags = len(tags)
tags

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data)
sent = getter.get_next()
print(sent)

In [ ]:
sentences = getter.sentences
print(len(sentences))
print(sentences)

In [ ]:
max_len = 50
X = [[w[0]for w in s] for s in sentences]
new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X.append(new_seq)
new_X[15]

In [ ]:
from keras.preprocessing.sequence import pad_sequences
tags2index = {t:i for i,t in enumerate(tags)}
y = [[tags2index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["o"])
y[15]

In [ ]:
from sklearn.model_selection import train_test_split
#import tensorflow as tf


import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
from keras import backend as K
X_tr, X_te, y_tr, y_te = train_test_split(new_X, y, test_size=0.2, random_state=2018)
sess = tf.Session()
K.set_session(sess)
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
batch_size = 2
def ElmoEmbedding(x):
    return elmo_model(inputs={"tokens": tf.squeeze(tf.cast(x,    tf.string)),"sequence_len": tf.constant(batch_size*[max_len])
                     },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [ ]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda
input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
batch_size = 2
X_tr, X_val = X_tr[:1213*batch_size], X_tr[-135*batch_size:]
y_tr, y_val = y_tr[:1213*batch_size], y_tr[-135*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)


In [ ]:
print(len(X_tr))
print(y_tr.shape)

In [ ]:
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())
saver.save(sess, 'my_test_model')

history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),batch_size=batch_size, epochs=3, verbose=1)

In [ ]:
!pip install seqeval

In [ ]:
print(len(X_te))

In [ ]:
X_back = X_te

In [ ]:
X_te = X_back

In [ ]:
#################NOT USED#####################
X_te = X_te[-90*batch_size:]

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print(len(X_te))
test_pred = model.predict(np.array(X_te), verbose=1, batch_size = 2)

In [ ]:
print(y_te.shape)

In [ ]:
idx2tag = {i: w for w, i in tags2index.items()}
def pred2label(ppred):
    out = []
    for pred_i in ppred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PADword", "o"))
        out.append(out_i)
    return out
def test2label(ppred):
    out = []
    for pred_i in ppred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p].replace("PADword", "o"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = test2label(y_te)
print(classification_report(test_labels, pred_labels))